## Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |

## Modelo de Programação Dinâmica para Estratégias Ótimas no Jogo "Quem Quer Ser um Milionário"

Neste problema, buscamos modelar o jogo Quem Quer Ser um Milionário? utilizando programação dinâmica para determinar estratégias ótimas que maximizem dois objetivos diferentes:

1) A recompensa esperada acumulada ao longo do jogo

2) A probabilidade de alcançar e responder corretamente a uma pergunta específica.

A solução envolve calcular, para cada estado do jogo (pergunta atual e ajudas disponíveis), as melhores decisões, considerando o uso ou não de ajudas e o impacto das probabilidades de sucesso.

### Resolução via Programação Dinâmica

#### Modelo de programação dinâmica

O modelo apresentado busca determinar as melhores decisões em cada etapa do jogo para atingir os dois objetivos descritos anteriormente.

Para isso, o modelo considera as recompensas associadas a cada pergunta, os riscos de errar e o impacto das ajudas disponíveis. A solução é calculada de forma recursiva, analisando todos os estados possíveis e as transições entre eles.

O modelo de programação dinâmica é apresentado abaixo.

* Estágios: cada estágio representa uma pergunta do jogo, numerada de 1 a num_perguntas. A lógica do modelo trabalha de trás para frente, ou seja, começa na última pergunta e calcula recursivamente o valor ótimo de cada estágio até a primeira pergunta.

* Estados: Um estado é representado pela pergunta atual k e pela disponibilidade das ajudas (ex: plateia, 50:50, telefone). As combinações das ajudas são modeladas como tuplas binárias, onde 1 significa que a ajuda está disponível, e 0 que já foi usada.

* Função de transição:

\begin{equation}
H(k,s) = \max\left\{r_k, max [p^s_a . H(k + 1, t(s,a)) + (1 - p^s_a) . r^´_k])\right\}
\end{equation}

Onde:

* r_k: Recompensa para desistir na pergunta k.
* A(s): Conjunto de todas as ações possíveis (combinações de uso das ajudas).
* P^s_a:  Probabilidade de sucesso ao tomar a ação "a" no estado "s".
* H(k+1,t(s,a)): Valor esperado no próximo estágio ao acertar a pergunta k.
* R'k: Recompensa ao errar a pergunta k.

Probabilidade de alcançar uma pergunta específica:

\begin{equation}
P(k,s) = \max\left\{[p^s_a . P(k + 1, t(s,a))\right\} \forall a \in A(s)
\end{equation}

#### Definição dos inputs

* num_perguntas: Número total de perguntas do jogo. Cada pergunta corresponde a um estágio no modelo. No exemplo, temos 15 perguntas.

* ajudas: Lista de ajudas disponíveis no jogo, como "plateia", "50:50" e "telefone". Isso permite modelar o impacto dessas opções nas probabilidades de sucesso.

* recompensas: Lista com os valores monetários atribuídos a cada pergunta. Por exemplo, a recompensa para a pergunta 1 é 100 e para a última é 1.000.000.

* recompensas_erro: Lista com os valores garantidos em caso de erro. Esses valores geralmente correspondem aos "pontos de segurança" do jogo, como 1000 ou 32000, que são preservados mesmo ao errar.

* probabilidades: Um dicionário que define:

  * base: Probabilidades de sucesso para cada pergunta, sem usar ajudas.

  * plateia, 50:50, telefone: Modificadores que aumentam as probabilidades de sucesso ao usar essas ajudas. Por exemplo, "plateia" multiplica a probabilidade base por 1.1.



#### Implementação


Implemente seu modelo na sequência. **A solução ótima é dada por $2$**, correspondente à subárvore $0,1,4,8,9$.

In [1]:
class JogoMilionarioDP:
    def __init__(self, num_perguntas, ajudas, recompensas, recompensas_erro, probabilidades):
        """
        Inicializa os parâmetros do modelo de programação dinâmica.

        Args:
            num_perguntas (int): Número total de perguntas no jogo.
            ajudas (list): Lista de ajudas disponíveis (ex.: ['plateia', '50:50', 'telefone']).
            recompensas (list): Lista de recompensas para cada pergunta.
            recompensas_erro (list): Recompensas garantidas ao errar.
            probabilidades (dict): Probabilidades de sucesso por pergunta e uso de ajudas.
        """
        self.num_perguntas = num_perguntas
        self.ajudas = ajudas
        self.recompensas = recompensas
        self.recompensas_erro = recompensas_erro
        self.probabilidades = probabilidades

    def maximizar_recompensa_esperada(self):
        """Calcula a recompensa esperada máxima usando programação dinâmica."""
        # Criação de uma matriz DP para armazenar os valores de H(v)
        dp = {}

        # Itera de trás para frente (da última pergunta até a primeira)
        for k in range(self.num_perguntas, 0, -1):
            for estado in self._gerar_estados():
                if k == self.num_perguntas:
                    # Última pergunta: calcular diretamente com as probabilidades disponíveis
                    dp[(k, estado)] = self._calcular_recompensa_terminal(k, estado)
                else:
                    dp[(k, estado)] = self._calcular_recompensa_recursiva(k, estado, dp)

        # Retorna a recompensa máxima esperada a partir da primeira pergunta
        return dp[(1, self._estado_inicial())]

    def maximizar_probabilidade_alcancar_pergunta(self, pergunta_alvo):
        """Maximiza a probabilidade de alcançar uma pergunta específica."""
        dp = {}

        # Itera de trás para frente (da pergunta_alvo até o início)
        for k in range(pergunta_alvo, 0, -1):
            for estado in self._gerar_estados():
                if k == pergunta_alvo:
                    # Alcançar a pergunta-alvo
                    dp[(k, estado)] = 1
                else:
                    dp[(k, estado)] = self._calcular_probabilidade_recursiva(k, estado, dp)

        return dp[(1, self._estado_inicial())]

    def _gerar_estados(self):
        """Gera combinações possíveis de estados (ajudas disponíveis)."""
        from itertools import product
        return list(product([0, 1], repeat=len(self.ajudas)))

    def _estado_inicial(self):
        """Estado inicial: todas as ajudas disponíveis."""
        return tuple([1] * len(self.ajudas))

    def _calcular_recompensa_terminal(self, k, estado):
        """Calcula a recompensa esperada para o estado terminal."""
        return max(self.recompensas[k - 1], self.recompensas_erro[k - 1])

    def _calcular_recompensa_recursiva(self, k, estado, dp):
        """Calcula a recompensa esperada para um estado intermediário."""
        max_recompensa = self.recompensas[k - 1]  # Caso de desistência

        # Testar todas as ações possíveis
        for acao_ajuda in self._gerar_acoes_ajudas(estado):
            prob_sucesso = self._obter_probabilidade(k, acao_ajuda)
            proximo_estado = self._transicao(estado, acao_ajuda)

            recompensa = (prob_sucesso * dp[(k + 1, proximo_estado)] +
                          (1 - prob_sucesso) * self.recompensas_erro[k - 1])
            max_recompensa = max(max_recompensa, recompensa)

        return max_recompensa

    def _calcular_probabilidade_recursiva(self, k, estado, dp):
        """Calcula a probabilidade máxima de alcançar a pergunta-alvo."""
        max_prob = 0

        for acao_ajuda in self._gerar_acoes_ajudas(estado):
            prob_sucesso = self._obter_probabilidade(k, acao_ajuda)
            proximo_estado = self._transicao(estado, acao_ajuda)
            max_prob = max(max_prob, prob_sucesso * dp[(k + 1, proximo_estado)])

        return max_prob

    def _gerar_acoes_ajudas(self, estado):
        """Gera combinações de uso de ajudas com base no estado atual."""
        from itertools import product
        return list(product([0, 1], repeat=len(self.ajudas)))

    def _obter_probabilidade(self, k, acao_ajuda):
        """Obtém a probabilidade de sucesso para uma ação específica."""
        prob_base = self.probabilidades['base'][k - 1]
        modificador = 1
        for i, uso in enumerate(acao_ajuda):
            if uso:
                modificador *= self.probabilidades[self.ajudas[i]][k - 1]
        return prob_base * modificador

    def _transicao(self, estado, acao_ajuda):
        """Transiciona para o próximo estado com base na ação atual."""
        return tuple(max(0, estado[i] - acao_ajuda[i]) for i in range(len(estado)))

# Exemplo de uso do modelo
jogo = JogoMilionarioDP(
    num_perguntas=15,
    ajudas=['plateia', '50:50', 'telefone'],
    recompensas=[100, 200, 300, 500, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 125000, 250000, 500000, 1000000],
    recompensas_erro=[0, 0, 0, 0, 1000, 1000, 1000, 1000, 1000, 32000, 32000, 32000, 32000, 32000, 32000],
    probabilidades={
        'base': [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2],
        'plateia': [1.1] * 15,
        '50:50': [1.2] * 15,
        'telefone': [1.15] * 15
    }
)

max_recompensa = jogo.maximizar_recompensa_esperada()
max_prob = jogo.maximizar_probabilidade_alcancar_pergunta(15)

print(f"Recompensa esperada máxima: {max_recompensa}")
print(f"Probabilidade máxima de alcançar a última pergunta: {max_prob}")


Recompensa esperada máxima: 119012.56132889456
Probabilidade máxima de alcançar a última pergunta: 0.056171321470692794
